# Task 2 - Forecasting

## Question 2 - Optimization Utilizing Forecasting
Suppose you have to decide at the beginning of each day the times at which you are going to charge and discharge, without any knowledge of the future, i.e., any price information of that day or future days. You do have information about past prices, however, which you may use to predict buy and sell periods. Can you design a strategy for charging and discharging under these circumstances? For this question, go back to the battery assumptions of 1.1, i.e. a 1MW, 1MWh battery.

Please explain your approach and in particular how you would train your trading strategy based on available information at any point in time. Illustrate your findings, and the difference to your results in question 1.1.



## Solution

One can use price forecast for the next day or hours to get better insight and support decision on when to buy or sell energy. Alternatively, based on the prediction (assuming we are confident with the results), we can then use the optimization functions to decide on the strategy to buy and sell. However, this strategy will only propagate what the forecast already tells us. Instead of using this two steps, one could model this problem as a classification, in which the model forecasts the action at each point in time. Effectivelly, forecast buy or sell. Obviously, this is a complex task since the model also need to understand the other constraints in which we can operate (simple example: is the battery empty or full?).

In any case, for this particular take home exercice, I will stick with forecasting price.

We want to predict `t+1` value based on `N` previous data points. For example, having close hourly prices from past days or even weeks, we want to predict what price will be on the next hour(s). Our Forecast Horizon will be the full next day.

I will train a set of models with different algorithm varing complexity to check which one would perform better in this experimental setting. 

I've stumbled upon a few recent articles proposing methods specificaly tailored for energy price forecasting, which alludes to the fact they are much better for this problem than the basic models I'm choosing for this exercise. 
The principles remains the same, but obviously with the caveats and intricacies of the specifics of each model.

As you will see at the end of this notebook, the basic model performs very poorly.

In scope:
* Linear regression, for simplicity
* Random Forest regressor
* Gradient Boosting
* ARIMA
* SARIMA
* TBATS - Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend and Seasonal components
* NeuralProphet

**Model Evaluation Metrics:**
* Mean Absolute Error (MAE): Measures the average magnitude of the errors without considering their direction. It's the simplest and most intuitive performance metric.
* Root Mean Squared Error (RMSE): Gives a relatively high weight to large errors. This means the RMSE should be more useful when large errors are particularly undesirable.

**Steps:**

1. Data preparation and data partition (train_data and test_data).
2. Model training and validation. Due to time and resources constraints, I will not perform cross-validation, but rather only train-test cycle. In production scenario I would consider cross-validation.
3. Forecast (with the 'best' performing model) ahead by a certain period of time for which you want to predict.

**TODOs**
* hyperparameter tuning and optimization
* implement the missing models


### References
- [An Intra-Day Electricity Price Forecasting Based on a Probabilistic Transformer Neural Network Architecture](https://www.researchgate.net/publication/374091409_An_Intra-Day_Electricity_Price_Forecasting_Based_on_a_Probabilistic_Transformer_Neural_Network_Architecture)
- [Short-Term Electricity Prices Forecasting Using Functional Time Series Analysis](https://www.mdpi.com/1996-1073/15/9/3423)
- [Forecasting day-ahead electricity prices: Utilizing hourly prices](https://www.sciencedirect.com/science/article/abs/pii/S0140988315001668)




In [ ]:
import sys
import os
import logging
from pathlib import PurePath
from environs import Env

In [ ]:
# add custom python modules root to the path variable, so that we can resuse code
root_path = PurePath(os.getcwd()).parents[1].joinpath("src")
if str(root_path) not in sys.path:
    sys.path.insert(0, str(root_path))
sys.path

In [ ]:
%load_ext autoreload
%autoreload 2

env = Env()
env.read_env(".env", override=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.getLogger("matplotlib").setLevel(
    logging.ERROR
)  # Only show errors for this EDA since matplotlib is too verbose. Don't do this in production ;)

dataset_path = env("ENERGY_TRADING_DATA")

In [ ]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import seasonal_decompose
from tbats import BATS, TBATS
from scipy import signal
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas.tseries.offsets import DateOffset

from challenges.energy_trading.data_access import load_energy_data
from typing import List, Tuple

myseed = 31
np.random.seed(myseed)
random.seed(myseed)

plt.rcParams.update({"figure.figsize": (10, 7), "figure.dpi": 200})
plt.style.use("seaborn-v0_8")

### Data preparation and Data partition

In [ ]:
def load_energy_data(filepath: str, suffix: str) -> pd.DataFrame:
    df = pd.read_csv(filepath + suffix, skiprows=1, names=["DateTime", "Price", "Currency"])
    return df


def propress_data(
    df,
    column: str,
    periods: int,
    date_fmt: str = "%d.%m.%Y %H:%M",
    decompose_model: str = "additive",
    decompose: bool = False,
):
    # Split the 'DateTime' column into 'start_time' and 'end_time'
    datetime_splits = df["DateTime"].str.split(" - ", expand=True)
    df["start_time"] = pd.to_datetime(datetime_splits[0], format=date_fmt)
    df["end_time"] = pd.to_datetime(datetime_splits[1], format=date_fmt)

    df = imput_missing_data(df, column)
    df = aggregate_timewindows(df)
    if decompose:
        offset = abs(df[column].min()) + 0.01  # offseting by the smallest value + 1 cent of EUR
        decomp = seasonal_decompose(df[column], period=periods, model=decompose_model, extrapolate_trend="freq")
        df[f"{column}_offset"] = df[column] + offset
        df[f"{column}_trend"] = decomp.trend
        df[f"{column}_seasonal"] = decomp.seasonal
        df[f"{column}_detrended"] = df[column] - decomp.trend
        df[f"{column}_detrended_lsf"] = signal.detrend(df[column])
        df[f"{column}_deseasonalized"] = df[column] - decomp.seasonal
        # differencing
        df[f"{column}_diff"] = df[column].diff().dropna()
        # log-transformed price, since we have no zeros in the Price column
        df[f"{column}_log"] = np.log(df[column] + offset)  # handling zeros: use a small offset constant
        # Seasonal differencing
        df[f"{column}_seasonal_diff"] = df[column].diff(
            periods
        )  # assuming 24 observations per day for daily seasonality
        # Combining Transformations:
        df[f"{column}_log_diff"] = np.log(df[column] + offset).diff().dropna()

    # clean up
    df = df.dropna()
    df = df.drop(columns=["DateTime", "end_time", "Currency"])
    df.set_index("start_time", inplace=True)

    return df


def aggregate_timewindows(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df["month"] = df.start_time.dt.month
    df["day_of_month"] = df.start_time.dt.day
    df["week"] = df.start_time.dt.isocalendar().week
    df["day_of_week"] = df.start_time.dt.isocalendar().day

    # Generate 'business hour' feature
    business_hours = ((df.start_time.dt.hour >= 7) & (df.start_time.dt.hour <= 12)) | (
        (df.start_time.dt.hour >= 14) & (df.start_time.dt.hour <= 19)
    )
    lunch_hours = (df.start_time.dt.hour > 12) & (df.start_time.dt.hour < 14)

    df["business_hours"] = 0  # Default to off hour
    df.loc[business_hours, "business_hours"] = 2  # Set to business hour
    df.loc[lunch_hours, "business_hours"] = 1  # Set to lunch hour

    # Generate 'weekend' feature
    df["weekend"] = 0  # Default to weekday
    df.loc[df.day_of_week == 5, "weekend"] = 1  # Set to Saturday
    df.loc[df.day_of_week == 6, "weekend"] = 2  # Set to Sunday
    return df


def imput_missing_data(df: pd.DataFrame, target_column: str) -> pd.DataFrame:
    # Check for any conversion issues
    if df[target_column].isnull().any():
        logging.warning(f"There are null values in the {target_column} column. Applying ffill.")
        df[target_column] = df[target_column].ffill()

        if pd.isna(df[target_column].iloc[0]):
            # Ensure no NaN values remain, if first value is NaN,
            # replace it with a sensible default (e.g., 0 or mean of the column)
            logging.warning("Imputing remaining missing values with median as fallback strategy.")
            df[target_column].iloc[0] = df[target_column].median()
    if df["Currency"].isnull().any():
        df["Currency"] = df["Currency"].fillna("EUR")
    return df


def prepare_daily_forecast_splits(
    df, test_size_percentage, n_splits, periods: int = 24
) -> List[Tuple[pd.DataFrame, pd.DataFrame]]:
    test_days = int((len(df) / periods) * test_size_percentage)
    test_size = test_days * periods

    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)

    splits = []
    for train_index, test_index in tscv.split(df):
        if test_index[0] % periods != 0:
            start_offset = periods - (test_index[0] % periods)
            test_index = range(test_index[0] + start_offset, test_index[-1] + 1)
        splits.append((df.iloc[train_index], df.iloc[test_index]))

    return splits

### Model training and validation

In [ ]:
def train_and_evaluate(models, splits):
    results = {}
    for model_name, model in models.items():
        maes = []
        rmses = []
        for X_train, X_test in splits:
            y_train, y_test = X_train["Price"], X_test["Price"]
            X_train, X_test = X_train.drop("Price", axis=1), X_test.drop("Price", axis=1)

            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

            maes.append(mean_absolute_error(y_test, predictions))
            rmses.append(np.sqrt(mean_squared_error(y_test, predictions)))

        results[model_name] = {"MAE": maes, "RMSE": rmses}
    return results

In [ ]:
# We also need to ensure there is not data leak in the training.
# Since we have created some features based on the target feature, we should remove those, which are:
# Price_offset, Price_trend, Price_seasonal, Price_detrended, Price_detrended_lsf, Price_deseasonalized, Price_diff, Price_log, Price_seasonal_diff, Price_log_diff
# These columns were useful during the EDA phase. For the training step we set decompose=False, so that these columsn are not included
e_prices_60m_df = propress_data(
    load_energy_data(dataset_path, "_60min.csv"), column="Price", periods=24, decompose=False
)

target_col = "Price"
features_cols = ["month", "day_of_month", "week", "day_of_week", "business_hours", "weekend"]

# Cross-validation
data_splits = prepare_daily_forecast_splits(
    e_prices_60m_df[features_cols + [target_col]], n_splits=5, test_size_percentage=0.1, periods=24
)
data_splits[0][0]

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100),
}

results = train_and_evaluate(models, data_splits)

In [ ]:
for model_name, metrics in results.items():
    print(f"Results for {model_name}:")
    for metric_name, values in metrics.items():
        print(f"\t{metric_name}: avg {np.mean(values):.2f}, std: {np.std(values):.2f}")

### Forecasting

Lets assume the `RandomForestRegressor` performed better than the others. 

At the final stage of model training, especially after selecting a model based on cross-validation results, it's common practice to train the model on the most comprehensive dataset available to maximize its learning and performance.

Thus, let's retrain the model and visualize the forecasting.


In [ ]:
full_training_data = e_prices_60m_df[features_cols + [target_col]]


# Define features and target
X = e_prices_60m_df[features_cols]
y = e_prices_60m_df[target_col]


# Splitting the data for final visualization (optional)
split_point = int(len(X) * 0.9)  # For example, hold out the last 10% of data
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

In [ ]:
# Train the Gradient Boosting Regressor on all available data
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)  # Assuming X_train and y_train include all data you plan to train on

# If you want to visualize how the model performs on this data:
predictions = model.predict(X_test)  # or on a new test set if available

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")

# Plotting the forecasts against the actual outcomes
plt.figure(figsize=(14, 7))
plt.plot(y_test.index, y_test, label="Actual Prices", color="blue", marker="o", linestyle="-")
plt.plot(y_test.index, predictions, label="Predicted Prices", color="red", marker="x", linestyle="--")
plt.title("Comparison of Actual and Predicted Prices")
plt.xlabel("Date")
plt.ylabel("Price (EUR/MWh)")
plt.legend()
plt.grid(True)
plt.show()